In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset, SubsetRandomSampler

from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
import itertools
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

Using device: cuda


In [22]:
NUM_EPOCHS = 80
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
BATCH_SIZE = 32
NUM_CLASSES = 10      # FashionMNIST has 10 classes
INPUT_CHANNELS = 1    # FashionMNIST is grayscale

In [23]:
image_transforms = transforms.Compose([
    #transforms.Resize(224)
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5], # Grayscale mean
        std=[0.5]   # Grayscale std
    ),
])

In [24]:
# Training
train_dataset = torchvision.datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform=image_transforms
)
# Testing
test_dataset = torchvision.datasets.FashionMNIST(
    root='./data',
    train=False,
    download=True,
    transform=image_transforms
)

# Data Loader
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=5
)
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=5
)

In [25]:
def train_model(model, train_loader, criterion, optimizer, num_epochs, device):
    """Handles the training process and evaluates after each epoch, tracking losses."""
    model.train()
    print("\nStarting training on FashionMNIST...")

    # Clear logs before starting
    train_losses, test_losses, test_accuracies=[],[],[]

    for epoch in range(num_epochs):
        total_loss = 0.0
        total_samples = 0

        # Training Loop
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # === sample-weighted loss, matches evaluation ===
            batch_size = labels.size(0)
            total_loss += loss.item() * batch_size
            total_samples += batch_size

        # === Correct per-sample training loss ===
        epoch_loss = total_loss / total_samples
        train_losses.append(epoch_loss)

        # Evaluate on test set
        acc, test_loss = evaluate_model(model, test_loader, criterion, device)

        test_losses.append(test_loss)
        test_accuracies.append(acc)

        print(f"Epoch {epoch+1} finished. Train Loss: {epoch_loss:.4f} | Test Loss: {test_loss:.4f} | Test Accuracy: {acc:.2f}%")
    return train_losses, test_losses, test_accuracies


def evaluate_model(model, test_loader, criterion, device):
    """Calculates the Top-1 accuracy AND the average test loss."""
    model.eval()
    correct_predictions = 0
    total_samples = 0
    total_loss = 0.0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            batch_size = labels.size(0)
            total_loss += loss.item() * batch_size
            total_samples += batch_size

            _, predicted = outputs.max(1)
            correct_predictions += (predicted == labels).sum().item()

    accuracy = 100 * correct_predictions / total_samples
    avg_loss = total_loss / total_samples

    return accuracy, avg_loss


In [26]:
class StandardCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),  # -> 28x28
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                                   # -> 14x14

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),  # -> 14x14
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                                     # -> 7x7
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),                     # 64 * 7 * 7 = 3136
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model=StandardCNN().to(DEVICE)

In [27]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(),lr=LEARNING_RATE,weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=NUM_EPOCHS, gamma=0.1)
import gc
gc.collect()
torch.cuda.empty_cache()
train_losses, test_losses, test_accuracies = train_model(
    model, train_loader, criterion, optimizer, NUM_EPOCHS, DEVICE
)



Starting training on FashionMNIST...


Epoch 1/80: 100%|██████████| 1875/1875 [00:10<00:00, 186.58it/s]


Epoch 1 finished. Train Loss: 0.6508 | Test Loss: 0.4356 | Test Accuracy: 84.20%


Epoch 2/80: 100%|██████████| 1875/1875 [00:09<00:00, 190.80it/s]


Epoch 2 finished. Train Loss: 0.3840 | Test Loss: 0.3736 | Test Accuracy: 86.80%


...

Epoch 78 finished. Train Loss: 0.0086 | Test Loss: 0.5345 | Test Accuracy: 91.78%


Epoch 79/80: 100%|██████████| 1875/1875 [00:09<00:00, 188.58it/s]


Epoch 79 finished. Train Loss: 0.0083 | Test Loss: 0.5518 | Test Accuracy: 91.75%


Epoch 80/80: 100%|██████████| 1875/1875 [00:09<00:00, 190.03it/s]


Epoch 80 finished. Train Loss: 0.0072 | Test Loss: 0.5634 | Test Accuracy: 91.76%


In [28]:
import pandas as pd
df = pd.DataFrame({
        "epoch": list(range(1, len(train_losses) + 1)),
        "train_loss": train_losses,
        "test_loss": test_losses,
        "test_accuracy": test_accuracies,
    })
df.to_csv("CNN_mnist.csv")